In [1]:
import pygame
from pygame.locals import *
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
# power-ups
# trump card
# increase bullet speed
# increase bullet capacity
# heal wall

(6, 0)

In [3]:
screenX = 1250
screenY = 800
win = pygame.display.set_mode((screenX, screenY))
pygame.display.set_caption("Trump Pepe 20")

bg = pygame.image.load("new background.png")
char = pygame.image.load("pepe sprite.png")
ok = pygame.image.load("ok.png")
trump = pygame.image.load("trump.png")
wall = pygame.image.load("wall.png")
trumpwall = pygame.image.load("trumpwall.png")

clock = pygame.time.Clock()

#bulletSound = pygame.mixer.Sound('bullet.wav')
#hitSound = pygame.mixer.Sound('hit.wav')
music = pygame.mixer.music.load('music.mp3')
pygame.mixer.music.play(-1)
score = 0

class player(object):
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.vel = 5
        self.isJump = False
        self.jumpCount = 10
        self.left = False
        self.right = False
        self.hitbox = (self.x + 70, self.y, self.w - 140, self.h)
        
    def draw(self, win):
        win.blit(char, (self.x, self.y))
        self.hitbox = (self.x + 70, self.y, self.w - 140, self.h)
        #pygame.draw.rect(win, (255, 0, 0), self.hitbox, 2)
    
    def hit(self):
        self.x -= 200
        font1 = pygame.font.SysFont('comicsans', 100)
        text1 = font1.render('You lost 10 votes', 1, (255, 255, 255))
        win.blit(text1, (screenX/2 - (text1.get_width()/2), screenY/2))
        pygame.display.update()
        i = 0
        while i < 300:
            pygame.time.delay(10)
            i += 10
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    i = 301
                    pygame.quit()

class projectile(object):
    def __init__(self, x, y, w, h, vel):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.vel = vel
    
    def draw(self, win, image):
        win.blit(image, (self.x, self.y))

class enemy(object):
    def __init__(self, x, y, w, h, vel, char, extra):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.vel = -vel
        self.char = char
        self.hitbox = (self.x, self.y, self.w, self.h)
        self.health = 10
        self.visible = True
        self.extra = extra
        
    def draw(self, win):
        self.move()
        if self.visible:
            win.blit(self.char, (self.x, self.y))
            self.hitbox = (self.x, self.y, self.w, self.h)
            #pygame.draw.rect(win, (255, 0, 0), self.hitbox, 2)
            pygame.draw.rect(win, (255, 0, 0), (self.hitbox[0], self.hitbox[1] - 20, 50, 10))
            pygame.draw.rect(win, (0, 255, 0), (self.hitbox[0], self.hitbox[1] - 20, 50 - (5 * (10 - self.health)), 10))

    def move(self):
        self.x += self.vel
        
    def hit(self):
        if self.health > 0:
            self.health -= 1
        else:
            self.visible = False
        #print('hit')

class theWall():
    img = trumpwall
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.hitbox = (self.x, self.y, self.w, self.h)
        self.health = 10
    
    def draw(self, win):
        win.blit(self.img, (self.x, self.y))
        if self.health >= 0:
            pygame.draw.rect(win, (255, 0, 0), (self.hitbox[0] + 50, self.hitbox[1] - 20, 50, 10))
            pygame.draw.rect(win, (0, 255, 0), (self.hitbox[0] +50, self.hitbox[1] - 20, 50 - (5 * (10 - self.health)), 10))

class button():
    def __init__(self, color, x, y, w, h, text=''):
        self.color = color
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.text = text
    
    def draw(self, win, outline=None):
        #call this method to draw the button on the screen
        if outline:
            pygame.draw.rect(win, outline, (self.x - 2, self.y - 2, self.w + 4, self.h + 4), 0)
        pygame.draw.rect(win, self.color, (self.x, self.y, self.w, self.h), 0)
        if self.text != '':
            font = pygame.font.SysFont('comicsans', 60)
            text = font.render(self.text, 1, (255, 255, 255))
            win.blit(text, (self.x + (self.w/2 - text.get_width()/2), self.y + (self.h/2 - text.get_height()/2)))
            
    def isOver(self, pos):
        #pos is the mouse position or a tuple of (x, y) coordinates
        if pos[0] > self.x and pos[0] < self.x + self.w:
            if pos[1] > self.y and pos[1] < self.y + self.h:
                return True
        return False
    
def redrawGameWindow():
    win.blit(bg, (0, 0))
    #blackButton.draw(win, (0, 0, 255))
    text = font.render('Votes: ' + str(score), 1, (255, 255, 255))
    win.blit(text, (screenX - text.get_width() - 100, 10))
    pepe.draw(win)
    ourWall.draw(win)
    for i in objects:
        i.draw(win)
    for bullet in bullets:
        bullet.draw(win, ok)
    for maga in magas:
        maga.draw(win, trump)
    for grab in grabs:
        grab.draw(win, wall)
    pygame.display.update()

def scoring():
    f = open("scores.txt", "r")
    file = f.readline()
    hiScore = int(file[0])
    f.close()
    return hiScore


def updateFile():
    global hiScore, score
    if hiScore < int(score):
        file = open("scores.txt", "w")
        file.write(str(score))
        file.close()

def endScreen():
    global bg, objects, rand1, rand2, bullets, magas, grabs, score, bulletloop, magaloop, ourWall, hiScore
    objects = []
    rand1 = 7000
    rand2 = 10000
    bulletloop = 0
    magaloop = 0
    bullets = []
    magas = []
    grabs = []
    run = True
    while run:
        pygame.time.delay(100)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                run = False
        win.blit(bg, (0, 0))
        largeFont = pygame.font.SysFont('comicsans', 80)
        if score > hiScore:
            high = largeFont.render("High Score: " + str(hiScore) + " Your Score: " + str(score) + " New High Score!", 1, (255, 255, 255))
            updateFile()
            hiScore = score
        else:
            high = largeFont.render("High Score: " + str(hiScore) + " Your Score: " + str(score), 1, (255, 255, 255))
        win.blit(high, ((screenX // 2) - (high.get_width() // 2), (screenY // 2) - (high.get_height() // 2)))
        pygame.display.update()
    score = 0
    ourWall.health = 10

    
#mainloop
rand1 = 7000
rand2 = 10000
pygame.time.set_timer(USEREVENT+2, random.randrange(rand1, rand2))
blackButton = button((0, 0, 0), 100, 100, 200, 200, 'HELP')
font = pygame.font.SysFont('comicsans', 64, True) #pygame fonts
pepe = player(screenX / 2, screenY - 224, 224, 224)
ourWall = theWall(0, screenY - 370, 224, 172)
bulletloop = 0
magaloop = 0
bullets = []
magas = []
grabs = []
objects = []
hiScore = scoring()

run = True
while run:
    clock.tick(27)
    for i in objects:
        if i.visible == False:
            objects.pop(objects.index(i))
    for i in objects:
        if i.visible == True:
            if pepe.hitbox[1] < i.hitbox[1] + i.hitbox[3] and pepe.hitbox[1] + pepe.hitbox[3] > i.hitbox[1]:
                if pepe.hitbox[0] + pepe.hitbox[2] > i.hitbox[0] and pepe.hitbox[0] < i.hitbox[0] + i.hitbox[2]:
                    pepe.hit()
                    score -= 10
            if ourWall.hitbox[1] < i.hitbox[1] + i.hitbox[3] and ourWall.hitbox[1] + ourWall.hitbox[3] > i.hitbox[1]:
                if ourWall.hitbox[0] + ourWall.hitbox[2] > i.hitbox[0] and ourWall.hitbox[0] < i.hitbox[0] + i.hitbox[2]:
                    ourWall.health -= 1
                    objects.pop(objects.index(i))
    if score < 0:
        endScreen()
    if ourWall.health < 0:
        endScreen()
    if bulletloop > 0:
        bulletloop += 1
    if bulletloop > 3:
        bulletloop = 0
    if magaloop > 0:
        magaloop += 1
    if magaloop > 5:
        magaloop = 0
    for event in pygame.event.get():
        pos = pygame.mouse.get_pos()
        if event.type == pygame.QUIT:
            run = False
        if event.type == USEREVENT+2:
            rand1 -= 10
            rand2 -= 10
            if score < 100:
                r = random.randrange(0, 5)
                if r == 0:
                    objects.append(enemy(screenX, screenY - 128, 128, 128, 3, pygame.image.load("democrat.png"), "dem"))
                elif r == 1:
                     objects.append(enemy(screenX, screenY - 128, 128, 128, 3, pygame.transform.scale(pygame.image.load("cnn.png"), (128, 128)), "cnn"))
                elif r == 2:
                    objects.append(enemy(screenX, screenY - 352, 128, 128, 3, pygame.image.load("mexican.png"), "mex"))
                elif r == 3:
                    objects.append(enemy(screenX, screenY - 352, 128, 128, 3, pygame.image.load("mexican.png"), "mex"))
                    objects.append(enemy(screenX, screenY - 128, 128, 128, 3, pygame.image.load("democrat.png"), "dem"))
                elif r == 4:
                    objects.append(enemy(screenX, screenY - 352, 128, 128, 3, pygame.image.load("mexican.png"), "mex"))
                    objects.append(enemy(screenX, screenY - 128, 128, 128, 3, pygame.transform.scale(pygame.image.load("cnn.png"), (128, 128)), "cnn"))
            else:
                for j in range(score//100):
                    r = random.randrange(0, 5)
                    vel = random.randrange(2, 9)
                    if r == 0:
                        objects.append(enemy(screenX, screenY - 128, 128, 128, vel, pygame.image.load("democrat.png"), "dem"))
                    elif r == 1:
                         objects.append(enemy(screenX, screenY - 128, 128, 128, vel, pygame.transform.scale(pygame.image.load("cnn.png"), (128, 128)), "cnn"))
                    elif r == 2:
                        objects.append(enemy(screenX, screenY - 352, 128, 128, vel, pygame.image.load("mexican.png"), "mex"))
                    elif r == 3:
                        objects.append(enemy(screenX, screenY - 352, 128, 128, vel, pygame.image.load("mexican.png"), "mex"))
                        objects.append(enemy(screenX, screenY - 128, 128, 128, vel, pygame.image.load("democrat.png"), "dem"))
                    elif r == 4:
                        objects.append(enemy(screenX, screenY - 352, 128, 128, vel, pygame.image.load("mexican.png"), "mex"))
                        objects.append(enemy(screenX, screenY - 128, 128, 128, vel, pygame.transform.scale(pygame.image.load("cnn.png"), (128, 128)), "cnn"))
        if event.type == pygame.MOUSEBUTTONDOWN:
            if blackButton.isOver(pos):
                print('HELP')
        if event.type == pygame.MOUSEMOTION:
            if blackButton.isOver(pos):
                blackButton.color = (255, 0, 0)
            else:
                blackButton.color = (0, 0, 0)
    for bullet in bullets:
        for i in objects:
            if i.extra == "cnn":   
                if i.visible == True:
                    if bullet.y - bullet.h < i.hitbox[1] + i.hitbox[3] and bullet.y + bullet.h > i.hitbox[1]:
                        if bullet.x + bullet.w > i.hitbox[0] and bullet.x - bullet.w < i.hitbox[0] + i.hitbox[2]:
                            i.hit()
                            score += 1
                            try:
                                bullets.pop(bullets.index(bullet))
                            except:
                                pass
        if bullet.x < screenX:
            bullet.x += bullet.vel
        else:
            try:
                bullets.pop(bullets.index(bullet))
            except:
                pass
    for maga in magas:
        for i in objects:
            if i.extra == "dem":
                if i.visible == True:
                    if maga.y - maga.h < i.hitbox[1] + i.hitbox[3] and maga.y + maga.h > i.hitbox[1]:
                        if maga.x + maga.w > i.hitbox[0] and maga.x - maga.w < i.hitbox[0] + i.hitbox[2]:
                            i.hit()
                            score += 1
                            try:
                                magas.pop(magas.index(maga))
                            except:
                                pass
        if maga.x < screenX:
            maga.x += maga.vel
        else:
            try:
                magas.pop(magas.index(maga))
            except:
                pass
    for grab in grabs:
        for i in objects:
            if i.extra == "mex":
                if i.visible == True:
                    if grab.y - grab.h < i.hitbox[1] + i.hitbox[3] and grab.y + grab.h > i.hitbox[1]:
                        if grab.x + grab.w > i.hitbox[0] and grab.x - grab.w < i.hitbox[0] + i.hitbox[2]:
                            i.hit()
                            score += 1
                            try:
                                grabs.pop(grabs.index(grab))
                            except:
                                pass
        if grab.x < screenX:
            grab.x += grab.vel
        else:
            try:
                grabs.pop(grabs.index(grab))
            except:
                pass
    keys = pygame.key.get_pressed()
    if keys[pygame.K_c]:
        if len(grabs) < 10:
            grabs.append(projectile(round(pepe.x + pepe.w // 2), round(pepe.y + pepe.h // 2), 32, 32, 50))
    elif keys[pygame.K_x] and magaloop == 0:
        if len(magas) < 2:
            magas.append(projectile(round(pepe.x + pepe.w // 2), round(pepe.y + pepe.h // 2), 32, 32, 20))
        magaloop = 1
    elif keys[pygame.K_v] and bulletloop == 0:
        if len(bullets) < 3:
            bullets.append(projectile(round(pepe.x + pepe.w // 2), round(pepe.y + pepe.h // 2), 64, 64, 10))
        bulletloop = 1
    if keys[pygame.K_LEFT] and pepe.x > pepe.vel:
        pepe.x -= pepe.vel
        pepe.left = True
        pepe.right = False
    elif keys[pygame.K_RIGHT] and pepe.x < screenX - pepe.w - pepe.vel:
        pepe.x += pepe.vel
        pepe.right = True
        pepe.left = False
    else:
        pepe.right = False
        pepe.left = False
        pepe.walkCount = 0
    if not(pepe.isJump):
        if keys[pygame.K_SPACE] or keys[pygame.K_UP]:
            pepe.isJump = True
            pepe.right = False
            pepe.left = False
            pepe.walkCount = 0
    else:
        if pepe.jumpCount >= -10:
            neg = 1
            if pepe.jumpCount < 0:
                neg = -1
            pepe.y -= (pepe.jumpCount ** 2) * .5 * neg
            pepe.jumpCount -= 1
        else:
            pepe.isJump = False
            pepe.jumpCount = 10
    redrawGameWindow()
pygame.quit()

In [5]:
objects[0]